## Instalando dependencias necesarias

In [1]:
%%capture
%pip install -U sentence-transformers pandas
from sentence_transformers import SentenceTransformer, util


## Entendiendo el dataset

In [ ]:
#Se descarga la base de datos de la siguiente ruta:
# https://www.kaggle.com/datasets/omarhanyy/imdb-top-1000?resource=download

In [2]:
#Si el archivo se carga en google drive conectas por medio del siguiente comando te conectas a tu cuenta
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#En caso que el archivo lo tengas en tu equipo local usa el siguiente Codigo para seleccionarlo
from google.colab import files
uploaded = files.upload()

In [3]:
import pandas as pd


In [4]:
# Se lee el archivo CSV y se crea el dataframe df
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Desarrollo_ProyectosIA/Proyecto1/IMDB top 1000.csv")

In [5]:
# se muestran los primeros 5 registros
df.head()

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info
0,0,1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80.0,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,...","Votes: 2,295,987 | Gross: $28.34M"
1,1,2. The Godfather (1972),R,175 min,"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime dyna...,Director: Francis Ford Coppola | Stars: Marlon...,"Votes: 1,584,782 | Gross: $134.97M"
2,2,3. The Dark Knight (2008),PG-13,152 min,"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,Director: Christopher Nolan | Stars: Christian...,"Votes: 2,260,649 | Gross: $534.86M"
3,3,4. The Godfather: Part II (1974),R,202 min,"Crime, Drama",9.0,90.0,The early life and career of Vito Corleone in ...,Director: Francis Ford Coppola | Stars: Al Pac...,"Votes: 1,107,253 | Gross: $57.30M"
4,4,5. The Lord of the Rings: The Return of the Ki...,PG-13,201 min,"Action, Adventure, Drama",8.9,94.0,Gandalf and Aragorn lead the World of Men agai...,"Director: Peter Jackson | Stars: Elijah Wood, ...","Votes: 1,614,369 | Gross: $377.85M"


In [ ]:
# Se valida el tamaño del dataframe en este caso tiene 1000 registros
df.shape

(1000, 10)

In [ ]:
# Se valida si existen registro duplicados
df.duplicated(df.columns[1])

0      False
1      False
2      False
3      False
4      False
       ...  
995     True
996     True
997     True
998     True
999     True
Length: 1000, dtype: bool

In [6]:
# Se eliminan los duplicados
df = df.drop_duplicates(df.columns[1])

In [7]:
# Se valida el tamaño del Dataframe despues de la eliminacion de duplicados
df.shape

(398, 10)

Se crea una nuevo columna **Busqueda** con la union de las columnas **Title**,**Description**,**cast**,**Genre** las cuales tienen informacion relevante como es el titulo de la pelicula, descripcion, director, actores y genero,  con el fin ampliar el espectro semantico para el modelo

In [8]:
df['Busqueda']= 'Titulo: '+df['Title']+' | '+'Description: '+df['Description']+'|'+df['Cast']+' | '+'Genre :'+df['Genre']

In [9]:
# Se valida la columna creada mostrando los primeros 5 registros
print(df.head())

   Unnamed: 0                                              Title Certificate  \
0           0                 1. The Shawshank Redemption (1994)           R   
1           1                            2. The Godfather (1972)           R   
2           2                          3. The Dark Knight (2008)       PG-13   
3           3                   4. The Godfather: Part II (1974)           R   
4           4  5. The Lord of the Rings: The Return of the Ki...       PG-13   

  Duration                     Genre  Rate  Metascore  \
0  142 min                     Drama   9.3       80.0   
1  175 min              Crime, Drama   9.2      100.0   
2  152 min      Action, Crime, Drama   9.0       84.0   
3  202 min              Crime, Drama   9.0       90.0   
4  201 min  Action, Adventure, Drama   8.9       94.0   

                                         Description  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

## Usando Sentence Transformer para crear embeddings

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#se vectoriza la columna Busqueda
embeddings = model.encode(df['Busqueda'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
#Se crea una nueva columna con los valores de la vectorizacion
df['embeddings'] = embeddings.tolist()

In [13]:
print(df.head())

   Unnamed: 0                                              Title Certificate  \
0           0                 1. The Shawshank Redemption (1994)           R   
1           1                            2. The Godfather (1972)           R   
2           2                          3. The Dark Knight (2008)       PG-13   
3           3                   4. The Godfather: Part II (1974)           R   
4           4  5. The Lord of the Rings: The Return of the Ki...       PG-13   

  Duration                     Genre  Rate  Metascore  \
0  142 min                     Drama   9.3       80.0   
1  175 min              Crime, Drama   9.2      100.0   
2  152 min      Action, Crime, Drama   9.0       84.0   
3  202 min              Crime, Drama   9.0       90.0   
4  201 min  Action, Adventure, Drama   8.9       94.0   

                                         Description  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

## Calculando la similitud usando la métrica de similitud por coseno

In [14]:
def compute_similarity(example, query_embedding):
    embedding = example['embeddings']
    similarity = util.cos_sim(embedding, query_embedding).item()
    return similarity


## Ejecuntando la búsqueda

In [15]:
while True:
    entrada = input("Introduce patron busqueda (o escribe 'salir' para terminar): ")

    if entrada.lower() == 'salir':  # Compara la entrada con 'salir' (sin importar mayúsculas)
        print("Programa finalizado.")
        break  # Sale del bucle si el usuario escribe 'salir'

    try:
        nombre = (entrada)  # Intenta convertir la entrada a un número
        query_embedding = model.encode(nombre)
        df['similarity'] = df.apply(lambda x: compute_similarity(x, query_embedding), axis=1)
        df = df.sort_values(by='similarity', ascending=False)
        print(df[['Title','Description','similarity']].head())
        # Aquí puedes añadir más lógica según lo que necesites hacer con el número
    except ValueError:
        print("Por favor, PATRON busqueda o 'salir'.")


Introduce patron busqueda (o escribe 'salir' para terminar): Matrix
                                        Title  \
15                      16. The Matrix (1999)   
251                   254. Underground (1995)   
309  312. Mr. Smith Goes to Washington (1939)   
331                    334. Special 26 (2013)   
157              158. A Beautiful Mind (2001)   

                                           Description  similarity  
15   When a beautiful stranger leads computer hacke...    0.380353  
251  A group of Serbian socialists prepares for the...    0.192562  
309  A naive man is appointed to fill a vacancy in ...    0.180245  
331  A gang of con-men rob prominent rich businessm...    0.176939  
157  After John Nash, a brilliant but asocial mathe...    0.166081  
Introduce patron busqueda (o escribe 'salir' para terminar): godfather
                                Title  \
1             2. The Godfather (1972)   
3    4. The Godfather: Part II (1974)   
108              109. Scarfac